In [1]:
import os
import gc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#!pip install prettytable
from collections import Counter
from prettytable import PrettyTable
from IPython.display import Image

from sklearn.preprocessing import LabelEncoder

from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.constraints import max_norm
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Dropout, Flatten, Activation, Reshape, Multiply
from keras.layers import Conv1D, Add, MaxPooling1D, BatchNormalization
from keras.layers import Embedding, Bidirectional, CuDNNLSTM, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras import layers
from sklearn.preprocessing import LabelEncoder
from keras.layers import concatenate
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
# Confusion Matrix
from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
# Precision
from sklearn.metrics import precision_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import sklearn.metrics
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
df_train = pd.read_pickle("train_seq.p")
df_test = pd.read_pickle("test_seq.p")
dt = pd.read_pickle("train_biological.p")
dk = pd.read_pickle("test_biological.p")

In [3]:
df_train = pd.concat([df_train,df_test], axis=0)
dt= pd.concat([dt,dk], axis=0)

In [3]:
k = pd.DataFrame()
k = df_train['Seq']

In [5]:
k.to_csv('seq.csv')

C:\Users\moham\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [3]:
df_3sn= pd.read_pickle("train_3s.p")
df_8sn = pd.read_pickle("train_8s.p")
df_3st = pd.read_pickle("test_3s.p")
df_8st = pd.read_pickle("test_8s.p")

In [4]:
df_3sn['solubility']=df_train['solubility']
df_8sn['solubility']=df_train['solubility']
df_3st['solubility']=df_test['solubility']
df_8st['solubility']=df_test['solubility']

In [6]:
del dk['Str8_H'] 
del dk['Str8_C'] 
del dk['Str8_S'] 
del dk['Str8_E'] 
del dk['Str8_T'] 
del dk['Str8_G'] 
del dk['Str8_B'] 
del dk['Str8_I'] 
del dk['Str3_H'] 
del dk['Str3_E'] 
del dk['Str3_C']

In [3]:
del dk['Seq']
del dk['3Str']
del dk['8Str']
del dt['Seq']
del dt['3Str']
del dt['8Str']
del dk['Acc_Sol']
del dt['Acc_Sol']

In [4]:
from sklearn import preprocessing   # normalized traing data set
x = dk.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()

x_scaled = min_max_scaler.fit_transform(x)
dk= pd.DataFrame(x_scaled)

x = dt.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()

x_scaled = min_max_scaler.fit_transform(x)
dt = pd.DataFrame(x_scaled)

In [5]:
X= dt.to_numpy()
X1 = dk.to_numpy()

In [6]:
def calc_unique_cls(train, test):
  """
  Prints # unique classes in data sets.
  """
  train_unq = np.unique(train['solubility'].values)
  #val_unq = np.unique(val['family_accession'].values)
  test_unq = np.unique(test['solubility'].values)

  #print('Number of unique classes in Train: ', len(train_unq))
  #print('Number of unique classes in Val: ', len(val_unq))
  #print('Number of unique classes in Test: ', len(test_unq))

In [7]:
df_train['seq_char_count']= df_train['Seq'].apply(lambda x: len(x))
#df_val['seq_char_count']= df_val['sequence'].apply(lambda x: len(x))
df_test['seq_char_count']= df_test['Seq'].apply(lambda x: len(x))

In [13]:
df_3st['seq_char_count']= df_3st['3Str'].apply(lambda x: len(x))
#df_val['seq_char_count']= df_val['sequence'].apply(lambda x: len(x))
df_8st['seq_char_count']= df_8st['8Str'].apply(lambda x: len(x))

In [14]:
df_3sn['seq_char_count']= df_3sn['3Str'].apply(lambda x: len(x))
#df_val['seq_char_count']= df_val['sequence'].apply(lambda x: len(x))
df_8sn['seq_char_count']= df_8sn['8Str'].apply(lambda x: len(x))

In [8]:
classes = df_train['solubility'].value_counts()[:2].index.tolist()
#len(classes)

In [9]:
train_sm = df_train.loc[df_train['solubility'].isin(classes)].reset_index()
#val_sm = df_val.loc[df_val['family_accession'].isin(classes)].reset_index()
test_sm = df_test.loc[df_test['solubility'].isin(classes)].reset_index()

#print('Data size after considering 1000 classes for each data split:')
#print('Train size :', len(train_sm))
#print('Val size :', len(val_sm))
#print('Test size :', len(test_sm))

In [17]:
train_sm1 = df_3sn.loc[df_3sn['solubility'].isin(classes)].reset_index()
#val_sm = df_val.loc[df_val['family_accession'].isin(classes)].reset_index()
test_sm1 = df_3st.loc[df_3st['solubility'].isin(classes)].reset_index()

In [18]:
train_sm2 = df_8sn.loc[df_8sn['solubility'].isin(classes)].reset_index()
#val_sm = df_val.loc[df_val['family_accession'].isin(classes)].reset_index()
test_sm2 = df_8st.loc[df_8st['solubility'].isin(classes)].reset_index()

In [19]:
train_sm2.head()

,index,8Str,solubility,seq_char_count
0,0,CCHHCCTTCCCHHHCCHHHHHHHHHHHHHHHHHCSCBCHHHHHHHS...,1,338
1,1,CEEEEEEEECSSCSBSCEEEEEEEEEGGGHHHHHHTTGGGCTTSCH...,1,230
2,2,CCEEEEEEEEECSTTCEEEEETTEEEEECCSCTTTCCSCCHHHHHH...,1,141
3,3,CCCHCCSEEEECSCCCTTSCEEECCECCCHHHHHHHHHHHHTSSSC...,1,193
4,4,CCCCEEECSSTTEEEEECSSSCEEEEEECSSTTCCSSSCCEEEETT...,1,230


In [10]:
codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

def create_dict(codes):
  char_dict = {}
  for index, val in enumerate(codes):
    char_dict[val] = index+1

  return char_dict

char_dict = create_dict(codes)

#print(char_dict)
#print("Dict Length:", len(char_dict))

In [21]:
codes1 = ['H','E','S','T','C','G','B','I']

def create_dict(codes1):
  char_dict1 = {}
  for index, val in enumerate(codes1):
    char_dict1[val] = index+1

  return char_dict1

char_dict1 = create_dict(codes1)

#print(char_dict)
#print("Dict Length:", len(char_dict))

In [22]:
codes2 = ['C','H','E']

def create_dict(codes2):
  char_dict2 = {}
  for index, val in enumerate(codes2):
    char_dict2[val] = index+1

  return char_dict2

char_dict2 = create_dict(codes2)

#print(char_dict)
#print("Dict Length:", len(char_dict))

In [11]:
def integer_encoding(data):
  """
  - Encodes code sequence to integer values.
  - 20 common amino acids are taken into consideration
    and rest 4 are categorized as 0.
  """
  
  encode_list = []
  for row in data['Seq'].values:
    row_encode = []
    for code in row:
      row_encode.append(char_dict.get(code, 0))
    encode_list.append(np.array(row_encode))
  
  return encode_list

In [24]:
def integer_encoding1(data):
  """
  - Encodes code sequence to integer values.
  - 20 common amino acids are taken into consideration
    and rest 4 are categorized as 0.
  """
  
  encode_list1 = []
  for row in data['3Str'].values:
    row_encode1 = []
    for code in row:
      row_encode1.append(char_dict1.get(code, 0))
    encode_list1.append(np.array(row_encode1))
  
  return encode_list1

In [25]:
def integer_encoding2(data):
  """
  - Encodes code sequence to integer values.
  - 20 common amino acids are taken into consideration
    and rest 4 are categorized as 0.
  """
  
  encode_list2 = []
  for row in data['8Str'].values:
    row_encode2 = []
    for code in row:
      row_encode2.append(char_dict2.get(code, 0))
    encode_list2.append(np.array(row_encode2))
  
  return encode_list2

In [12]:
train_encode = integer_encoding(train_sm) 
#val_encode = integer_encoding(val_sm) 
test_encode = integer_encoding(test_sm)

In [28]:
train_encode1 = integer_encoding1(train_sm1) 
#val_encode = integer_encoding(val_sm) 
test_encode1 = integer_encoding1(test_sm1)

In [29]:
train_encode2 = integer_encoding2(train_sm2) 
#val_encode = integer_encoding(val_sm) 
test_encode2 = integer_encoding2(test_sm2)

In [32]:
# padding sequences

max_length =100
train_pad = pad_sequences(train_encode, maxlen=max_length, padding='post', truncating='post')
#val_pad = pad_sequences(val_encode, maxlen=max_length, padding='post', truncating='post')
test_pad = pad_sequences(test_encode, maxlen=max_length, padding='post', truncating='post')

#train_pad.shape, test_pad.shape

In [47]:
# padding sequences

max_length =80
train_pad1 = pad_sequences(train_encode1, maxlen=max_length, padding='post', truncating='post')
#val_pad = pad_sequences(val_encode, maxlen=max_length, padding='post', truncating='post')
test_pad1 = pad_sequences(test_encode1, maxlen=max_length, padding='post', truncating='post')

#train_pad.shape, test_pad.shape

In [48]:
# padding sequences

max_length =80
train_pad2 = pad_sequences(train_encode2, maxlen=max_length, padding='post', truncating='post')
#val_pad = pad_sequences(val_encode, maxlen=max_length, padding='post', truncating='post')
test_pad2 = pad_sequences(test_encode2, maxlen=max_length, padding='post', truncating='post')

#train_pad.shape, test_pad.shape

In [33]:
train_ohe = to_categorical(train_pad)
#val_ohe = to_categorical(val_pad)
test_ohe = to_categorical(test_pad)

#train_ohe.shape, test_ohe.shape

In [50]:
train_ohe1 = to_categorical(train_pad1)
#val_ohe = to_categorical(val_pad)
test_ohe1 = to_categorical(test_pad1)

#train_ohe.shape, test_ohe.shape

In [51]:
train_ohe2 = to_categorical(train_pad2)
#val_ohe = to_categorical(val_pad)
test_ohe2 = to_categorical(test_pad2)

#train_ohe.shape, test_ohe.shape

In [34]:
# label/integer encoding output variable: (y)
le = LabelEncoder()

y_train_le = le.fit_transform(train_sm['solubility'])
#y_val_le = le.transform(val_sm['family_accession'])
y_test_le = le.fit_transform(test_sm['solubility'])

#y_train_le.shape, y_test_le.shape

In [35]:
# One hot encoding of outputs
y_train = to_categorical(y_train_le)
#y_val = to_categorical(y_val_le)
y_test = to_categorical(y_test_le)

#y_train.shape, y_test.shape

In [27]:
# Utility function: plot model's accuracy and loss

# https://realpython.com/python-keras-text-classification/
plt.style.use('ggplot')

def plot_history(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  x = range(1, len(acc) + 1)

  plt.figure(figsize=(12, 5))
  plt.subplot(1, 2, 1)
  plt.plot(x, acc, 'b', label='Training acc')
  plt.plot(x, val_acc, 'r', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()

  plt.subplot(1, 2, 2)
  plt.plot(x, loss, 'b', label='Training loss')
  plt.plot(x, val_loss, 'r', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

In [28]:
def display_model_score(model, train, val, test, batch_size):

  train_score = model.evaluate(train[0], train[1], batch_size=batch_size, verbose=1)
  print('Train loss: ', train_score[0])
  print('Train accuracy: ', train_score[1])
  print('-'*70)

  val_score = model.evaluate(val[0], val[1], batch_size=batch_size, verbose=1)
  print('Val loss: ', val_score[0])
  print('Val accuracy: ', val_score[1])
  print('-'*70)
  
  test_score = model.evaluate(test[0], test[1], batch_size=batch_size, verbose=1)
  print('Test loss: ', test_score[0])
  print('Test accuracy: ', test_score[1])

In [29]:
def SE_block(input_shape):
    x = GlobalMaxPooling1D()(input_shape)
    x = Dense(int(int(input_shape.shape[-1])/8),activation='relu')(x)
    x = Dense(int(input_shape.shape[-1]),activation='sigmoid')(x)
    x = Reshape((-1,int(input_shape.shape[-1])))(x)
    #x = Concatenate()([x, input_shape])
    x = Multiply()([x, input_shape])
    return x

In [30]:
def residual_block(data, filters, d_rate):
  """
  _data: input
  _filters: convolution filters
  _d_rate: dilation rate
  """

  shortcut = data

  bn1 = BatchNormalization()(data)
  act1 = Activation('relu')(bn1)
  conv1 = Conv1D(filters, 1, dilation_rate=d_rate, padding='same', kernel_regularizer=l2(0.001))(act1)

  #bottleneck convolution
  bn2 = BatchNormalization()(conv1)
  act2 = Activation('relu')(bn2)
  conv2 = Conv1D(filters, 3, padding='same', kernel_regularizer=l2(0.001))(act2)
  

  #bn3 = BatchNormalization()(conv2)
  #act3 = Activation('relu')(bn3)
  #conv3 = Conv1D(filters, 4, padding='same', kernel_regularizer=l2(0.001))(act3)
  conv = SE_block(conv2)
  #skip connection
  x = Add()([conv, shortcut])

  return x

In [31]:
kfold = 10


random_state = 1
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []
save_dir = '/saved_models/'
t = np.zeros(kfold)
#a = np.zeros(kfold)
b = np.zeros(kfold)
#c = np.zeros(kfold)
#d = np.zeros(kfold)


skf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=random_state)
k = 0
epochs =50
batch_size =128

for train_index, test_index in skf.split(train_ohe, y_train_le):
    X_train, X_test = train_ohe[train_index], train_ohe[test_index]
   # X_train3, X_test3 = train_ohe1[train_index], train_ohe1[test_index]
    #X_train4, X_test4 = train_ohe2[train_index], train_ohe2[test_index]
    X_train2, X_test2 = X[train_index], X[test_index]
    y_train, y_test = y_train_le[train_index], y_train_le[test_index]

    optimizer = Adam(lr=0.001, decay=1e-8)
    x_input = Input(shape=(800, 21))
    #x_input1 = Input(shape=(80,6))
    #x_input2 = Input(shape=(80,4))
    input2 = Input(shape=(62,))

#initial conv
    conv = Conv1D(128, 1, padding='same')(x_input) 

# per-residue representation
    res1 = residual_block(conv, 128, 2)
    res2 = residual_block(res1, 128, 3)

    x = MaxPooling1D(3)(res2)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

# softmax classifier
    x = Flatten()(x)
    #conv1 = Conv1D(20, 1, padding='same')(x_input1) 

# per-residue representation
    #res11 = residual_block(conv1, 20, 2)
    #res21 = residual_block(res11, 20, 3)

   # x1 = MaxPooling1D(3)(res21)
   # x1 = BatchNormalization()(x1)
    #x1 = Dropout(0.5)(x1)

# softmax classifier
    #x1 = Flatten()(x1)
    #conv2 = Conv1D(20, 1, padding='same')(x_input2) 

# per-residue representation
    #res12 = residual_block(conv2, 20, 2)
    #res22 = residual_block(res12, 20, 3)

    #x2 = MaxPooling1D(3)(res22)
    #x2 = BatchNormalization()(x2)
    #x2 = Dropout(0.5)(x2)

# softmax classifier
    #x2 = Flatten()(x2)
    x = concatenate([x, input2])
    #x1= concatenate([x1, x])
    #x2= concatenate([x2,x1])
    x_output = Dense(1, activation='sigmoid', kernel_regularizer=l2(0.0001))(x)
 
    model2 = Model(inputs=[x_input,input2], outputs=x_output)
    model2.compile(optimizer=optimizer, loss= 'binary_crossentropy', metrics=['accuracy'])
    es = EarlyStopping(monitor='loss_accuracy', patience=5, verbose=1)
    #model2.summary()
    his = model2.fit([X_train,X_train2], y_train ,batch_size=batch_size,  epochs=epochs, validation_data= ([X_test,X_test2], y_test),verbose=1,  shuffle=True)


    y_pred = model2.predict([test_ohe, X1]).round().astype(int)
    y_train_pred = model2.predict([X_train, X_train2]).round().astype(int)

    t[k] = sklearn.metrics.f1_score(y_test_le, y_pred, average='weighted')
    # Confusion Matrix
    #print(t[k], b[k])
# Accuracy
    b[k] = accuracy_score(y_test_le, y_pred)
    print(t[k], b[k])
# Recall
    #c[k] = recall_score(y_test, y_pred, average=None)
    model2.save('model4_'+str(k)+".h5")
    results = model2.evaluate([test_ohe, X1], y_test_le)
    results = dict(zip(model2.metrics_names,results))
    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])
# Precision

    k+=1
    
print ('Average f1 score', np.mean(t))
print ('Average ac score', np.mean(b))


Train on 10194 samples, validate on 1134 samples
Epoch 1/50


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv1d_6/convolution (defined at C:\Users\moham\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_7254]

Function call stack:
keras_scratch_graph
